<a href="https://colab.research.google.com/github/rudycav/RedditScraper/blob/edit/Project/r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import praw
from praw.models import MoreComments
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
import re
import string
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
import requests

url = 'https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

for start in range(1, 100, 1000):

    params = {
        'start': start,
        'limit': 500,
    }

    r = requests.get(url, params=params)
    data = r.json()

ticker = [(crypto['symbol'].lower()) for crypto in (data['data'])]
name_lower = [(crypto['name'].lower()) for crypto in (data['data'])]


In [4]:
reddit = praw.Reddit(
    client_id="BE3T0CMpNrtqLT97yL6XiA",
    client_secret = "gJnvUXTK0gCnj3UMGSUtw0JW35tN1A",
    password = "Hannah00!!@",
    user_agent = "spidey by u/rudyboycav",
    username = "rudyboycav",
    check_for_async=False
)

In [5]:
def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for x in hot_topic: 
    c = x.comments.list()
    for y in c:
      if isinstance(y, MoreComments):
        continue
      df = df.append({'title': x.title,
                      'author': y.author,
                      'comments': y.body,
                      'upvote_score': y.score},                                          
                      ignore_index=True) 

  return df


In [6]:
wall = reddit_scraper(reddit, 'CryptoCurrency', 2)

In [7]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenizer = RegexpTokenizer(r'\w+')
sia = SIA()

def process_text(headlines):
    tokens = []
    for line in headlines:
        toks = tokenizer.tokenize(line)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    
    return tokens

In [8]:
'''
sia = SIA()
ticker_mentions = []
name_mentions = []
ff = []

#ticker_mentions = [pol_score = sia.polarity_scores(y) for y in wall.comments for x in ticker if x in y]


for x in ticker:
  for y in wall.comments:
    if x in y:
      if y not in ticker_mentions:
        ticker_mentions.append(y)
      else:
        ticker_mentions = ticker_mentions
'''       
          




'\nsia = SIA()\nticker_mentions = []\nname_mentions = []\nff = []\n\n#ticker_mentions = [pol_score = sia.polarity_scores(y) for y in wall.comments for x in ticker if x in y]\n\n\nfor x in ticker:\n  for y in wall.comments:\n    if x in y:\n      if y not in ticker_mentions:\n        ticker_mentions.append(y)\n      else:\n        ticker_mentions = ticker_mentions\n'

In [9]:
results = []
for x in wall.comments:
    pol_score = sia.polarity_scores(x)
    pol_score['replies'] = x
    results.append(pol_score)


In [10]:
df2 = pd.DataFrame.from_records(results)

df2['label'] = 0
df2.loc[df2['compound'] > 0.2, 'label'] = 1
df2.loc[df2['compound'] < -0.2, 'label'] = -1

df2.head()

,neg,neu,pos,compound,replies,label
0,0.134,0.866,0.000,-0.9167,Bullish factors for 2023:\n\n- we already drop...,-1
1,0.055,0.797,0.147,0.7418,I just need to be filled with hopium for this ...,1
2,0.033,0.886,0.081,0.8244,I am willing to bet the fed pivots this year a...,1
3,0.036,0.838,0.125,0.9468,My outlook is incredibly basic but I genuinely...,1
4,0.000,0.912,0.088,0.5578,Long slow year ahead. Load up your bags if you...,1


In [ ]:
pos_lines = list(df2[df2.label == 1].replies)

pos_tokens = process_text(pos_lines)
#pos_freq = nltk.FreqDist(pos_tokens)

#pos_freq.most_common(20)
pos_tokens

In [ ]:
neg_lines = list(df2[df2.label == -1].replies)

neg_tokens = process_text(neg_lines)
#pos_freq = nltk.FreqDist(pos_tokens)

#pos_freq.most_common(20)
neg_tokens

In [ ]:
neu_lines = list(df2[df2.label == 0].replies)

neu_tokens = process_text(neu_lines)
#pos_freq = nltk.FreqDist(pos_tokens)

#pos_freq.most_common(20)
neu_tokens

In [14]:
vv = []
dc = []
nn = []

for x in pos_tokens:
  for y in ticker:
    if x == y:
      vv.append(y)

for x in neg_tokens:
  for y in ticker:
    if x == y:
      dc.append(y)

for x in neu_tokens:
  for y in ticker:
    if x == y:
      nn.append(y)

In [34]:
ccc = Counter(dc)
cccc = ccc.most_common(6)
df3 = pd.DataFrame.from_records(list(dict(cccc).items()), columns=['page','count'])
df3

TypeError: ignored

In [32]:
ggg = Counter(nn)

gggg =ggg.most_common(6)

In [33]:
vfv = Counter(vv)
vfvv = vfv.most_common(6)

In [16]:
for x in name:
  for y in wall.comments:
    if x in y:
      if y not in name_mentions:
          pol_score = sia.polarity_scores(y)
          pol_score['replies'] = y
          name_mentions.append(pol_score)

NameError: ignored

In [ ]:
name_mentions

In [ ]:
li = {}
dd = []

for s in ticker:
  for x in wall.comments:
    if s in x:
      
      if s not in li:
 
        li[s] = 1
      else:
        li[s] += 1

In [ ]:
li

In [ ]:
for x in wall.comments:
  ff = x.split()
  pp = list(set(filter(lambda ff: ff.lower().startswith('$'), ff)))
  output = re.sub(r'\d+', '', str(pp))


  if len(pp) > 0:
    print(output)




In [ ]:
for x in tokey:
  print(x.startswith('$'))

In [ ]:
for x in xx:
  print(x)

In [ ]:
def clean_text(df):
  clean_text = " ".join(str(df).split())
  clean_text = " ".join([w for w in df.split() if w.isalpha()])

  return clean_text

gg = wall.comments.map(clean_text)

In [ ]:
for x in gg:
  print(x)

In [ ]:
def word_tokenizer(df):
  tokens = word_tokenize(str(df))

  return tokens

ggg =word_tokenizer(gg)

In [ ]:
print(ggg)
print(len(ggg))

In [ ]:
stopwords_ = set(stopwords.words("english"))

In [ ]:

def stopwords(df):
  clean_tokens = [t for t in df if not t in stopwords_]
  clean_text = " ".join(clean_tokens)

  return clean_tokens

gggg = stopwords(ggg)

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('cats')

In [ ]:
for x in gggg:
  ff = lemmatizer.lemmatize(x)
  print(x)

In [ ]:
def lemmatizer(df):
  return [lemmatizer.lemmatize(word) for word in df]
gggg.map(str(lemmatizer))


In [ ]:
print(gggg)
print(len(gggg))

In [ ]:
non_letter = "".join([x for x in str(cleaned_text) if x not in string.punctuation])
non_letter

In [ ]:
def remove_punctuation(df):

  #non_letter = re.sub(r"https?://\S+", "", str(df))
  non_letter = "".join([x for x in str(df) if x not in string.punctuation]) #removes punctuation
  non_letter = ''.join([i for i in str(df) if not i.isdigit()]) #removes numbers
  #non_letter = demojize(str(df))
  
  return non_letter

cleaned_text = remove_punctuation(cleaned_text)

In [ ]:
cleaned_text

In [ ]:
for x in cleaned_text:
  if x is 'SPY':
    print(x)

In [ ]:
for x in cleaned_text:
  if x.isupper():
    print(x)

In [ ]:
def tokenize(df):
  for words in wall:
    tokenize = word_tokenize(words)
  return tokenize

tokenized_words = tokenize(wall)

In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
def stopwords(df):
  #stop_words = set(stopwords.words("english"))
  filtered_words = [word for word in df if word.casefold() not in stop_words]
  return filtered_words

filtered_stopwords = stopwords(tokenized_words)
filtered_stopwords

In [ ]:
def stemming(df):
  stemmed_list = []
  snow_stemmer = SnowballStemmer(language='english')
  
  for words in filtered_stopwords:
    stemmer = snow_stemmer.stem(words)
    stemmed_list.append(stemmer)
  return stemmed_list

stemmed_words = stemming(filtered_stopwords)
stemmed_words

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets")
async for x in subreddit.hot(limit=3):
    print('THREAD: {}'.format(x.title))
    print('USER: {}'.format(x.author))

    print(x.comments)
    

